# 数据载入

In [38]:
import pandas as pd
import os
from collections import Counter
import numpy as np

import torch

In [39]:
pd.set_option('display.max_colwidth', -1)

C:\Users\c'y'y\AppData\Local\Temp\ipykernel_8392\3652881321.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [40]:
poi_path =r'E:\xpj\research\POI\psqlcsv\bangkok.csv'
poi_prompt_path =r'E:\xpj\research\POI\result\bangkok_poi_prompt.xlsx'
image_path = r'E:\xpj\research\POI\psqlcsv\bangkok_img_location.csv'
image_poi_path = r'E:\xpj\research\POI\psqlcsv\bangkok_img_poi.csv'
desc_path =r'E:\xpj\research\POI\desc_merge\bangkok_desc_merged.csv'
best_poi_path = r'E:\xpj\research\POI\result\bangkok_best_poi.xlsx'
desc_poi_path = r'E:\xpj\research\POI\result\bangkok_desc_poi.xlsx'

## poi数据（全面）

In [41]:
poi_data = pd.read_csv(poi_path,encoding='utf-8')
# poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_poi.csv",encoding='ANSI')
poi_df = pd.DataFrame(poi_data)

d:\software\Anaconda\envs\clip\lib\site-packages\IPython\core\interactiveshell.py:3269: DtypeWarning: Columns (7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [42]:
poi_df

,id,geom,osm_id,name,barrier,highway,ref,address,is_in,place,man_made,other_tags
0,1,0101000020E61000001DDF837C752659403311DB824A512B40,61703359,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0101000020E61000000C4169030B265940F01A4F5F2A492B40,61703363,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
2,3,0101000020E61000006C7DEC890929594060DE8893A0782B40,61705883,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
3,4,0101000020E610000045323F92ED23594030BB270F0B792B40,61709729,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,NaN,NaN
4,5,0101000020E6100000E16EFF80BD2459409615D4015A942B40,61711121,รัชดา-สุทธิสาร,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""day_off""=>""saturday"",""day_on""=>""monday"",""hour_off""=>""22:00"",""hour_on""=>""14:00"",""junction""=>""yes"",""name:en""=>""Rachada-Suthisarn"""
...,...,...,...,...,...,...,...,...,...,...,...,...
88162,88163,0101000020E6100000AB9E82B25F2B59404C04053983462B40,11381174879,NaN,yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88163,88164,0101000020E6100000BC95253ACB2A59400EE83BAEFC502B40,11381220183,บ้านส้มตำ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""air_conditioning""=>""yes"",""amenity""=>""restaurant"",""cuisine""=>""thai"",""name:en""=>""Baan Somtum"",""name:th""=>""บ้านส้มตำ"",""opening_hours""=>""Mo-Su 11:00-22:00"""
88164,88165,0101000020E6100000F5AD201766205940291197D2E9872B40,11381348042,Kope Hya Tai Kee @ Prachathipatai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""addr:city""=>""แขวงบางขุนพรหม"",""addr:housenumber""=>""110"",""addr:postcode""=>""10200"",""addr:street""=>""ถนนประชาธิปไตย"",""amenity""=>""restaurant"",""cuisine""=>""coffee_shop"",""opening_hours""=>""7.000-20.00"",""outd"
88165,88166,0101000020E6100000EF82D261CF1F59403D29931ADA842B40,11381886969,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""shop""=>""hairdresser"""


In [43]:
poi_df.columns

Index(['id', 'geom', 'osm_id', 'name', 'barrier', 'highway', 'ref', 'address',
       'is_in', 'place', 'man_made', 'other_tags'],
      dtype='object')

 # 下面第一块可以不做

In [7]:
# save tag list to txt

# poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway','ref', 'address', 'place', 'man_made', 'other_tags']]
# poi_df_ot =poi_df_nec.loc[:,['other_tags']]
# tag_list = []
# def collect_tags(x):
#     # print(x['other_tags'])
    
#     if  pd.notna(x['other_tags']):
#         # print(x['other_tags'])
#         tags = x['other_tags'].split("\",\"")
#         # print(tags)
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 if t not in tag_list:
#                     tag_list.append(t)
#             except:
#                 pass
#     # return tag_list

# result = poi_df_ot.apply(lambda x: collect_tags(x), axis=1, result_type='expand')
# # result
# tag_list.sort()
# tag_list
# # 保存tag_list到txt
# with open(r'E:\xpj\research\POI\chennai_tag_list.txt', 'w') as f:
#     for item in tag_list:
#         f.write("%s\n" % item)

# poi_df_ot

# def genSent(x):
#     tags = None
#     b = True    
#     # sen = "The imformation of this point contains that "
#     sen = "Nearby,"
#     for key in ['name','barrier', 'highway','address','place','man_made']:
#         if not x[key].isna().iloc[0]:
#             # print(x[key])
#             sen = sen + key +" is " + x[key].iloc[0] + ","
#     if not x['other_tags'].isna().iloc[0]:
#         tags = x['other_tags'].iloc[0].split("\",\"")
#         for tag in tags:
#             try:
#                 tag = tag.replace("\"","")
#                 t = tag.split("=>")[0]
#                 k = tag.split("=>")[1]
#                 if "name:" in t and t != "name:en":
#                     pass
#                 sen = sen + t +" is " + k + ","
#             except:
#                 pass

#     sen = sen + "located in " + x['NAME_2'].iloc[0]
    
#     # return sen,b
    
#     return sen
# # poi_df_nec


## 照片信息

In [44]:
pic_data = pd.read_csv(image_path,encoding='utf-8')
# pic_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_points.csv",encoding='ANSI')
pic_df = pd.DataFrame(pic_data)

d:\software\Anaconda\envs\clip\lib\site-packages\IPython\core\interactiveshell.py:3269: DtypeWarning: Columns (13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [45]:
pic_df

,id,geom,captured_a,camera_mak,created_at,sequences_,pano,user_key,username,img_key,ca,longitude,latitude,img_id,lon,lat,layer,path
0,180437,0101000020E610000076AB425EEA265940B8F99001EA7F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,JA9VFkde-r9epYaKeoUdaw,225.664,100.608055,13.749832,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
1,180438,0101000020E6100000E2A65027EA2659400CD01154E87F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,YWepTGuhB9Q3f5F5dzMcyw,224.522,100.608042,13.749819,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
2,180439,0101000020E61000001702CAF0E92659400DA837A3E67F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,oYDB3rBsEXmqPf1nifvWRA,224.522,100.608029,13.749807,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
3,180440,0101000020E61000004D5D43BAE9265940617EB8F5E47F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,5IijP-Idkn8tt6V18cGkmQ,224.522,100.608016,13.749794,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
4,180441,0101000020E610000083B8BC83E92659406256DE44E37F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,Yk-qJEgwMKnAzUpeR9JZ2A,224.522,100.608003,13.749781,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015899,180433,0101000020E610000088FC8E3CEB26594079AF5A99F07F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,T3WzU3WaFaSYCq4dYqxRGg,225.664,100.608108,13.749883,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
1015900,180434,0101000020E610000029983105EB265940C780ECF5EE7F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,wMCovOXqKrXkfjzBqiEd3g,225.664,100.608095,13.749870,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
1015901,180435,0101000020E6100000FFD368CDEA265940C253234FED7F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,m28js8eug9o0MPxhDMK8cg,225.664,100.608081,13.749857,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp
1015902,180436,0101000020E6100000D50FA095EA265940BD265AA8EB7F2B40,2018/07/22 05:52:53.000,NaN,2018/08/04 08:08:04.000,pOR29yQ1QJe4DAgNekGERw,0,ux-SzTkD96XBfXW04kBerw,toffy,mc7mEJ_dMm2JAXF5EG2Tyw,225.664,100.608068,13.749845,NaN,NaN,NaN,Bangkok_img_location,C:/Users/24491/Documents/FileRecv/region/mapillary_img_location/Bangkok_img_location.shp


## 照片poi关联

In [46]:
pic_poi_data = pd.read_csv(image_poi_path,encoding='utf-8')
# pic_poi_data = pd.read_csv(r"E:\xpj\research\POI\mumbai_pois_to_points.csv",encoding='ANSI')
pic_poi_df = pd.DataFrame(pic_poi_data)

In [47]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5
0,000KITttaQnmeNsgcwT_PA,45400,6.924021,58623.0,34.832125,45401.0,42.176896,75979.0,43.370676,68932.0,62.684223
1,000MZKvGMeqW_dPgcjIL3Q,43367,98.803905,51727.0,102.109848,428.0,161.992859,306.0,182.505537,72.0,184.847919
2,000oOGuhzsjec9uV71G1Ig,73330,25.306178,73331.0,44.202560,45691.0,166.877640,30438.0,276.295206,58336.0,285.590105
3,000oRp3E5uFaK1UjpjfCDA,65415,9.594348,49496.0,12.858370,56793.0,23.749958,827.0,27.180638,56759.0,27.700707
4,000q_gZY_eJyop5akxKjjA,44244,53.613539,35488.0,53.882897,44242.0,252.151236,35486.0,293.183568,44282.0,333.354862
...,...,...,...,...,...,...,...,...,...,...,...
884115,zZzwpESoUO9lwq_WIF9bmg,73327,174.724397,73329.0,188.890789,73328.0,190.624950,73333.0,251.807144,43629.0,279.793364
884116,ZzZXZKTEd_YiOFo9CDChgA,14828,129.643955,28702.0,146.434939,74206.0,155.462364,45355.0,405.171300,NaN,NaN
884117,zZZykOeloUZ6-f-4R7tdtg,48327,52.521084,40963.0,133.839863,16704.0,176.684535,66564.0,182.706714,1316.0,203.839085
884118,zzZytpKByu5aiE_SgGWCVA,64119,34.255010,47849.0,57.774322,48248.0,78.100325,47848.0,121.275198,54040.0,149.280831


## 照片描述

In [48]:
desc_pd =pd.read_csv(desc_path,encoding='utf-8')
desc_pd

,key,prompt
0,--9rgYB2ZCHkuLTrlOlplA,"there is a small building with a fence and a fire hydrant, a detailed matte painting inspired by Gao Cen, behance, paris school, modern lush condo as shopfront, ouchh and and innate studio, lecture halls and gambling dens"
1,--WYjbVJIq5eUrNyjjxAew,"there is a narrow street with a white brick wall and a blue and white wall, a detailed matte painting inspired by Albert Anker, pixiv, barbizon school, white block fence, found on google street view, narrow footpath"
2,-0CVvVBkckomDYWJLelOHg,"cars are driving on the road near a city with tall buildings, a tilt shift photo inspired by Ned M. Seidler, featured on pexels, hypermodernism, digital billboard in the middle, found on google street view, electronic billboards"
3,-0Ti_T4nO59b0V-lrzkGUw,"arafed street with a fence and a car parked on the side, a matte painting inspired by Thomas Struth, shutterstock contest winner, barbizon school, unobstructed road, google maps street view, narrow footpath"
4,-1Pca6jyCveNvlqippwRaQ,"there is a bus that is driving down the street in the city, a tilt shift photo inspired by Gao Cen, polycount, paris school, lecture halls and gambling dens, narrow footpath, unobstructed road"
...,...,...
1005899,lWYx1OMYiv0tIn8aC0qFiQ,"arafed view of a highway with a truck driving on it, a matte painting by Gao Cen, behance, tonalism, heavy rain approaching, dark cloudy sky, bad weather approaching"
1005900,trlZkSgxggFMxu2_GGBcRA,"arafly shot of a highway with cars and a bridge, a matte painting by Kume Keiichiro, tumblr, video art, car on highway, dashcam footage, highway"
1005901,qW-9JShsv4O6tMej0p1aWQ,"cars driving on a highway with a bridge in the background, a matte painting inspired by Friedrich Traffelet, polycount, superflat, dash cam, dashcam footage, dash cam footage"
1005902,WgA-pNLZz10kIsqAt7mFEg,"arafed view of a car driving down a street with a bridge, a screenshot by Rube Goldberg, featured on reddit, superflat, dashcam footage, car jumping, dash cam footage"


# 数据处理

## poi数据预处理

In [49]:
poi_df_nec=poi_df.loc[:,['id', 'name', 'barrier', 'highway', 'address', 'place', 'man_made', 'other_tags']]

In [50]:
poi_df_nec

,id,name,barrier,highway,address,place,man_made,other_tags
0,1,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal"""
3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN
4,5,รัชดา-สุทธิสาร,NaN,NaN,NaN,NaN,NaN,"""day_off""=>""saturday"",""day_on""=>""monday"",""hour_off""=>""22:00"",""hour_on""=>""14:00"",""junction""=>""yes"",""name:en""=>""Rachada-Suthisarn"""
...,...,...,...,...,...,...,...,...
88162,88163,NaN,yes,NaN,NaN,NaN,NaN,NaN
88163,88164,บ้านส้มตำ,NaN,NaN,NaN,NaN,NaN,"""air_conditioning""=>""yes"",""amenity""=>""restaurant"",""cuisine""=>""thai"",""name:en""=>""Baan Somtum"",""name:th""=>""บ้านส้มตำ"",""opening_hours""=>""Mo-Su 11:00-22:00"""
88164,88165,Kope Hya Tai Kee @ Prachathipatai,NaN,NaN,NaN,NaN,NaN,"""addr:city""=>""แขวงบางขุนพรหม"",""addr:housenumber""=>""110"",""addr:postcode""=>""10200"",""addr:street""=>""ถนนประชาธิปไตย"",""amenity""=>""restaurant"",""cuisine""=>""coffee_shop"",""opening_hours""=>""7.000-20.00"",""outd"
88165,88166,NaN,NaN,NaN,NaN,NaN,NaN,"""shop""=>""hairdresser"""


In [ ]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec.csv")

# 硬提示计算相似度，找最佳poi

In [51]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

# 加载BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model = AutoModel.from_pretrained(r"E:\xpj\models\bert-base-uncased")
model.to(device)

In [52]:
# 定义计算相似度的函数
def calc_similarity(s1, s2):
    # 对句子进行分词，并添加特殊标记
    inputs = tokenizer([s1, s2], return_tensors='pt', padding=True, truncation=True)
    inputs.to(device)
    # print(inputs)

    # 将输入传递给BERT模型，并获取输出
    with torch.no_grad():
        outputs = model(**inputs)
        embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()

    # 计算余弦相似度，并返回结果
    sim = np.dot(embeddings[0], embeddings[1]) / (np.linalg.norm(embeddings[0]) * np.linalg.norm(embeddings[1]))
    return sim


In [53]:

def isEnglish(s):
    if isinstance(s, str):
        return s.isascii()
    return False



# 从txt中读取tag_list
tag_ignore_list = []
with open(r'E:\xpj\research\POI\tag_ignore_list.txt', 'r') as f:
    for line in f.readlines():
        tag_ignore_list.append(line.strip())
# print(tag_ignore_list)

def genSent(x):
    tags = None
    b = True    
    # sen = "The imformation of this point contains that "
    sen = "Nearby,"
    for key in ['name','barrier', 'highway','address','place','man_made']:
        if isinstance(x[key],str) and not isEnglish(x[key]):
            continue
        # # 北京保留中文
        # if key == 'name' and not isEnglish(x['name'].iloc[0]):
        #     continue
        if not x[key].isna().iloc[0]:
            # print(x[key])
            sen = sen + key +" is " + x[key].iloc[0] + ","
    # print(sen)
    if not x['other_tags'].isna().iloc[0]:
        tags = x['other_tags'].iloc[0].split("\",\"")
        for tag in tags:
            try:
                tag = tag.replace("\"","")
                t = tag.split("=>")[0]
                k = tag.split("=>")[1]
                # 北京保留中文
                if not isEnglish(k) or not isEnglish(t):
                    print(k,t)
                    continue
                if list(t)[-1] == "=":
                    continue
                if t in tag_ignore_list:
                    continue
                # if "name:" in t and t != "name:en":
                #     pass
                # pass 的作用是什么也不做 跳过，continue的作用是执行下次for循环
                # print(t,k)
                sen = sen + t +" is " + k + ","
            except:
                pass

    # sen = sen + "located in " + x['NAME_2'].iloc[0]
    
    # return sen,b
    
    return sen

In [ ]:
# a="脿陇陇脿陇戮脿陇聹脿陇庐脿陇鹿脿陇戮脿陇脿陇陋脿楼聟脿陇虏脿楼聡脿陇脿陇鹿脿楼聣脿陇聼脿楼聡脿陇"
# # print(isEnglish(a))
# b="à¤­à¤¾à¤°à¤¤à¥à¤à¤¹à¤µà¤¾à¤®à¤¾à¤¨ à¤µà¤¿à¤­à¤¾à¤"
# print(isEnglish(b))
# print(b.encode('utf-8').decode('ISO-8859-1'))
# s = "啶ぞ啶班い啷€啶啶灌さ啶距ぎ啶距え 啶掂た啶ぞ啶"
# print(s.encode('ansi').decode('iscii'))

将poi对应的自然语言提示存储到文件中(poi_nec最后插入一列提示)

In [54]:
def store_poi(x):
    # print(x)
    poi = x[0]
    # print(poi)    
    poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
    sen = genSent(poi_im)
    return sen
    # print(sen)


    

In [55]:
from tqdm import tqdm
result =[]
for index, row in tqdm(poi_df_nec.iterrows(), total=poi_df_nec.shape[0], desc="Processing"):
    # if index == 130:
    #     print(store_poi(row))
    result.append(store_poi(row))
# result = poi_df_nec.apply(lambda x: store_poi(x), axis=1, result_type='expand')
# poi_df_nec['poi_prompt']= result
# poi_df_nec['poi_prompt'] = result

Processing:   0%|          | 134/88167 [00:00<02:08, 684.65it/s]

แยกมิตรสัมพันธ์ name:th
ด่านเก็บค่าผ่านทางพิเศษ ดินแดง official_name
ด่านเก็บค่าผ่านทางพิเศษ บางนา official_name
แยกราชวิถี name:th
ด่านเก็บค่าผ่านทางพิเศษ ประชาชื่น (ขาออก) official_name
สามเสน name:th
สส railway:ref
ด่านเก็บค่าผ่านทางพิเศษ ดินแดง 1 official_name
ด่านเก็บค่าผ่านทางพิเศษ อาจณรงค์ 1 (บางนา) official_name
ด่านเก็บค่าผ่านทางพิเศษ ประชาชื่น (ขาเข้า) official_name
ท่าช้าง name:th
ด่านเก็บค่าผ่านทางพิเศษ ดาวคะนอง official_name
ด่านเก็บค่าผ่านทางพิเศษ อโศก 1 official_name
ด่านเก็บค่าผ่านทางพิเศษ อโศก 4 official_name
คลองพุทรา name:th
พระแก้ว name:th
มาบกะเบา name:th
ผาเสด็จ name:th
ท่าเรือวิบูลย์ศรี name:th
ทางแยกประสพสุข name:th
พิษณุโลก name:th
ลพบุรี name:th
왓 아룬 가는 나루; cross rhe river description
טאן ט name:he
หนองศาลา name:th
ศาลายา name:th
เขาพนมแบก name:th
th:สถานีรถไฟเขาพนมแบก wikipedia
ท่าชนะ name:th
เขาสวนทุเรียน name:th
บ้านกรูด name:th
แยกสี่พระยา name:th
th:แยกสี่พระยา wikipedia
แขวงสุริยวงศ์ addr:city
ถนนสีลม addr:street
สุรศักดิ์ name:th
ช่องนนทรี name:th
ราชดำ

Processing:   0%|          | 287/88167 [00:00<01:59, 736.14it/s]

เชียงรากน้อย name:th
เชียงราก name:th
ดอนเมือง name:th
หลักสี่ name:th
บางเขน name:th
นิคมรถไฟ กม.11 name:th
th:ป้ายหยุดรถไฟนิคมรถไฟ กม.11 wikipedia
จิตรลดา name:th
จล railway:ref
นครปฐม name:th
คฐ. railway:ref
ราชบุรี name:th
รร. railway:ref
เพชรบุรี name:th
ประจวบคีรีขันธ์ name:th
Чумпхон name:ru
ชุมพร name:th
ปากตะโก name:th
ด่านเก็บค่าผ่านทางพิเศษสะพานสว่าง name:th
ด่านเก็บค่าผ่านทางพิเศษ สะพาน official_name
ウィセット・チャイシー門 name:ja
แยกสี่เสาเทเวศร์ name:th
พระปิ่นเกล้า name:th
แยกการเรือน name:th
ท่าเรือไอคอนสยาม 1 name:th
ราชเทวี name:th
พญาไท name:th
อนุสาวรีย์ชัยสมรภูมิ name:th
สนามเป้า name:th
อารีย์ name:th
เสนาร่วม name:th
サパーンクワイ name:ja
สะพานควาย name:th
モーチット name:ja
หมอชิต name:th
นานา name:th
ชิดลม name:th
ด่านเก็บค่าผ่านทางพิเศษ อโศก 3 official_name
แยกเทคนิคกรุงเทพ name:th
プロムポン name:ja
พร้อมพงษ์ name:th
แยกสวัสดี name:th
アソーク駅 name:ja
아소크 name:ko
อโศก name:th
แยกอโศก alt_name
เพลินจิต name:th
ทองหล่อ name:th
Эккамай name:ru
พระโขนง name:th
อ่อนนุช name:th
หัวลำโพง name:t

Processing:   0%|          | 440/88167 [00:00<01:59, 734.41it/s]

...

สมอลล์เวิลด์ name:th
สำนักงานองค์การนิสิต มหาวิทยาลัยศรีนครินทรวิโรฒ full_name
ถ้ำแม่พระ name:th
อนุสาวรีย์ศาสตราจารย์ดร.สาโรช บัวศรี name:th
อนุสาวรีย์ศาสตราจารย์ม.ล.ปิ่น มาลากุล name:th
ถนนคอนแวนด์ name:th
ช้างหมอบ name:th
Sip N‘ Camp name:en
บ้านครัวใต้ name:th
บ้านครัวเหนือ name:th


Processing: 100%|█████████▉| 88041/88167 [01:58<00:00, 751.32it/s]

ศูนย์ราชการนนทบุรี name:th
ศูนย์ราชการนนทบุรี name:th
แคราย name:th
แคราย name:th
สนามบินน้ำ name:th
สามัคคี name:th
กรมชลประทาน name:th
ศรีรัช name:th
ศรีรัช name:th
เมืองทองธานี name:th
เมืองทองธานี name:th
แจ้งวัฒนะ 14 name:th
แจ้งวัฒนะ 14 name:th
ศูนย์ราชการเฉลิมพระเกียรติ name:th
ศูนย์ราชการเฉลิมพระเกียรติ name:th
โทรคมนาคมแห่งชาติ name:th
โทรคมนาคมแห่งชาติ name:th
หลักสี่ name:th
วัดพระศรีมหาธาตุ name:th
ราชภัฏพระนคร name:th
รามอินทรา 3 name:th
ลาดปลาเค้า name:th
รามอินทรา กม.4 name:th
มัยลาภ name:th
วัชรพล name:th
รามอินทรา กม.6 name:th
รามอินทรา กม.9 name:th
คู้บอน name:th
วงแหวนรามอินทรา name:th
นพรัตน์ name:th
เบอร์เกอร์วิทยา name:th
ซอยศาลาแดง 3 addr:street
ซอยศาลาแดง 3 addr:street
ซอยศาลาแดง 3 addr:street
ซอยศาลาแดง 3 addr:street
ซอยศาลาแดง 3 addr:street
ซอยศาลาแดง 3 addr:street
ถนนศาลาแดง addr:street
ถนนศาลาแดง addr:street
ถนนศาลาแดง addr:street
ถนนศาลาแดง addr:street
ถนนศาลาแดง addr:street
แขวงเสนานิคม addr:city
ชอนมัน name:th
เซ็นบัน name:th
จ๊อดแฟร์ แดนเนรมิต name:th
โร

Processing: 100%|██████████| 88167/88167 [01:58<00:00, 741.85it/s]

สยาม สเตเดียม โฮสเทล name:th
ประชาคมเจริญผล name:th
หุ่นยนต์ทรานส์ฟอร์มเมอร์ส name:th
เขียง name:th
อีเปีย name:th
คิดNap name:th
บ้านส้มตำ name:th
แขวงบางขุนพรหม addr:city
ถนนประชาธิปไตย addr:street


In [56]:
import pandas as pd
print(pd.options)

In [57]:
# poi_df_nec.to_csv(r"E:\xpj\research\POI\poi_df_nec3.csv")
poi_df_nec['poi_prompt'] = result
poi_df_nec.to_excel(poi_prompt_path)
# poi_df_nec.to_csv(poi_prompt_path)


In [58]:
poi_df_nec =pd.read_excel(poi_prompt_path)
poi_df_nec

,Unnamed: 0,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,0,1,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
1,1,2,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
2,2,3,NaN,NaN,traffic_signals,NaN,NaN,NaN,"""traffic_signals""=>""signal""","Nearby,highway is traffic_signals,traffic_signals is signal,"
3,3,4,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
4,4,5,รัชดา-สุทธิสาร,NaN,NaN,NaN,NaN,NaN,"""day_off""=>""saturday"",""day_on""=>""monday"",""hour_off""=>""22:00"",""hour_on""=>""14:00"",""junction""=>""yes"",""name:en""=>""Rachada-Suthisarn""","Nearby,name is รัชดา-สุทธิสาร,day_off is saturday,day_on is monday,hour_off is 22:00,hour_on is 14:00,junction is yes,name:en is Rachada-Suthisarn,"
...,...,...,...,...,...,...,...,...,...,...
88162,88162,88163,NaN,yes,NaN,NaN,NaN,NaN,NaN,"Nearby,barrier is yes,"
88163,88163,88164,บ้านส้มตำ,NaN,NaN,NaN,NaN,NaN,"""air_conditioning""=>""yes"",""amenity""=>""restaurant"",""cuisine""=>""thai"",""name:en""=>""Baan Somtum"",""name:th""=>""บ้านส้มตำ"",""opening_hours""=>""Mo-Su 11:00-22:00""","Nearby,name is บ้านส้มตำ,air_conditioning is yes,amenity is restaurant,cuisine is thai,name:en is Baan Somtum,"
88164,88164,88165,Kope Hya Tai Kee @ Prachathipatai,NaN,NaN,NaN,NaN,NaN,"""addr:city""=>""แขวงบางขุนพรหม"",""addr:housenumber""=>""110"",""addr:postcode""=>""10200"",""addr:street""=>""ถนนประชาธิปไตย"",""amenity""=>""restaurant"",""cuisine""=>""coffee_shop"",""opening_hours""=>""7.000-20.00"",""outd","Nearby,name is Kope Hya Tai Kee @ Prachathipatai,amenity is restaurant,cuisine is coffee_shop,"
88165,88165,88166,NaN,NaN,NaN,NaN,NaN,NaN,"""shop""=>""hairdresser""","Nearby,shop is hairdresser,"


In [59]:
def find_best_poi(x):
    # print(x)
    img_key = x['img_key']
    # img_file = img_key+'.jpg'
    img_file =img_key
    try:
        img_desc = desc_pd.loc[desc_pd['key']==img_file]['prompt'].iloc[0]
    except:
        print("The description of picture " + img_file +" doesn't exist")
        return  None,None
    pois = x[1::2]
    pois_dis =x[2::2]
    # print(pois)
    similarities = []
    sentences = []
    for index,poi in enumerate(pois):
        # 设置距离阈值
        if pois_dis[index] is not None and pois_dis[index]<100:

            # # 根据id从poi_df_nec中找到poi_prompt
            try:
                sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
            except Exception as e:
                print(e)
                print("The poi_prompt of poi " + poi +" doesn't exist")
                continue
            
            # poi_im = poi_df_nec.loc[poi_df_nec['id']==poi]
            # sen = genSent(poi_im)
            # try:
            # # 检查 DataFrame 是否为空
            #     sen = poi_df_nec.loc[poi_df_nec['id']==poi]['poi_prompt'].iloc[0]
            #     # 在此处理 sen 的其他操作

            # except Exception as e:
            #     print(e)
            #     # print("An error occurred while processing poi " + poi)
            #     continue

        
            similarity = calc_similarity(img_desc,sen)
            sentences.append(sen)
            similarities.append(similarity)
        
    if len(similarities) == 0:
        best_poi=None
        return best_poi,0
    elif similarities and max(similarities)>0.5:
        best_poi = pois[similarities.index(max(similarities))]
        return best_poi,max(similarities)

In [60]:
result = []
for _, row in tqdm(pic_poi_df.iterrows(), total=pic_poi_df.shape[0], desc="Processing"):
    result.append(find_best_poi(row))

# pic_poi_df[['best_poi', 'best_poi_score']] = result

Processing:   0%|          | 126/884120 [00:26<42:50:37,  5.73it/s]

The description of picture 00oQfwiVjXdO-4-gANJlBw doesn't exist


Processing:   0%|          | 138/884120 [00:28<36:20:51,  6.76it/s]

The description of picture 00Q701jhc-1ncBJbfSlgtg doesn't exist


Processing:   0%|          | 233/884120 [00:50<46:32:07,  5.28it/s]

...

Processing: 100%|█████████▉| 883920/884120 [59:51:45<00:20,  9.78it/s]

The description of picture zzTpQ437715krz9YtuBk4A doesn't exist


Processing: 100%|█████████▉| 883922/884120 [59:51:46<00:23,  8.27it/s]

The description of picture ZztQGeet__Qggfwgk-OEhA doesn't exist


Processing: 100%|██████████| 884120/884120 [59:52:24<00:00,  4.10it/s]


In [61]:
result

[(58623.0, 0.7186676),
 (43367, 0.81200165),
 (73330, 0.6630048),
 (827.0, 0.77517587),
 (35488.0, 0.74152184),
 (40501.0, 0.79780835),
 (72820, 0.7187543),
 (58810.0, 0.73085654),
 (58849, 0.7604931),
 (None, 0),
 (47964.0, 0.7930419),
 (None, 0),
 (87838, 0.7562126),
 (None, 0),
 (9437, 0.7820624),
 (55911.0, 0.7944723),
 (306.0, 0.7553101),
 (35358.0, 0.7105006),
 (86732, 0.759052),
 (236.0, 0.677507),
 (4916, 0.7857914),
 (69941, 0.8091932),
 (68064.0, 0.795344),
 (69078.0, 0.7876377),
 (77622, 0.6417832),
 (50047.0, 0.6345037),
 (357, 0.78273964),
 (33712.0, 0.8006897),
 (7822.0, 0.7662196),
 (29095.0, 0.82809526),
 (2380, 0.7939934),
 (19880, 0.7069635),
 (5635, 0.7700389),
 (6963.0, 0.744937),
 (67361, 0.7717745),
 (47725.0, 0.7148908),
 (87481, 0.6549274),
 (31658.0, 0.7981422),
 (None, 0),
 (69110, 0.7730378),
 (None, 0),
 (34070, 0.6205337),
 (18196.0, 0.7641576),
 (4839, 0.78284633),
 (9665, 0.71128654),
 (2991, 0.6522284),
 (None, 0),
 (48984.0, 0.7763678),
 (None, 0),
 (69

In [62]:
result
result_df = pd.DataFrame(result,columns=['best_poi','best_poi_score']) # 将列表转换成DataFrame
pic_poi_df = pic_poi_df.join(result_df) # 将新的DataFrame添加到原始DataFrame中
# pic_poi_df.shape[0]

In [ ]:
# 上一块做了，这一块就不做了
#result = pic_poi_df.apply(lambda x: find_best_poi(x), axis=1,result_type='expand')

In [ ]:
#pic_poi_df[['best_poi','similarity']]= result

In [63]:
pic_poi_df

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,distance5,best_poi,best_poi_score
0,000KITttaQnmeNsgcwT_PA,45400,6.924021,58623.0,34.832125,45401.0,42.176896,75979.0,43.370676,68932.0,62.684223,58623.0,0.718668
1,000MZKvGMeqW_dPgcjIL3Q,43367,98.803905,51727.0,102.109848,428.0,161.992859,306.0,182.505537,72.0,184.847919,43367.0,0.812002
2,000oOGuhzsjec9uV71G1Ig,73330,25.306178,73331.0,44.202560,45691.0,166.877640,30438.0,276.295206,58336.0,285.590105,73330.0,0.663005
3,000oRp3E5uFaK1UjpjfCDA,65415,9.594348,49496.0,12.858370,56793.0,23.749958,827.0,27.180638,56759.0,27.700707,827.0,0.775176
4,000q_gZY_eJyop5akxKjjA,44244,53.613539,35488.0,53.882897,44242.0,252.151236,35486.0,293.183568,44282.0,333.354862,35488.0,0.741522
...,...,...,...,...,...,...,...,...,...,...,...,...,...
884115,zZzwpESoUO9lwq_WIF9bmg,73327,174.724397,73329.0,188.890789,73328.0,190.624950,73333.0,251.807144,43629.0,279.793364,NaN,0.000000
884116,ZzZXZKTEd_YiOFo9CDChgA,14828,129.643955,28702.0,146.434939,74206.0,155.462364,45355.0,405.171300,NaN,NaN,NaN,0.000000
884117,zZZykOeloUZ6-f-4R7tdtg,48327,52.521084,40963.0,133.839863,16704.0,176.684535,66564.0,182.706714,1316.0,203.839085,48327.0,0.766675
884118,zzZytpKByu5aiE_SgGWCVA,64119,34.255010,47849.0,57.774322,48248.0,78.100325,47848.0,121.275198,54040.0,149.280831,47849.0,0.747958


In [64]:
pic_poi_df.to_excel(best_poi_path)

In [65]:
df1 = pd.merge(pic_poi_df, poi_df_nec, left_on="best_poi",right_on = 'id',how="left")

In [66]:
df1

,img_key,id1,distance1,id2,distance2,id3,distance3,id4,distance4,id5,...,Unnamed: 0,id,name,barrier,highway,address,place,man_made,other_tags,poi_prompt
0,000KITttaQnmeNsgcwT_PA,45400,6.924021,58623.0,34.832125,45401.0,42.176896,75979.0,43.370676,68932.0,...,58622.0,58623.0,NaN,lift_gate,NaN,NaN,NaN,NaN,"""access""=>""permissive""","Nearby,barrier is lift_gate,"
1,000MZKvGMeqW_dPgcjIL3Q,43367,98.803905,51727.0,102.109848,428.0,161.992859,306.0,182.505537,72.0,...,43366.0,43367.0,NaN,NaN,NaN,NaN,NaN,NaN,"""noexit""=>""yes""","Nearby,noexit is yes,"
2,000oOGuhzsjec9uV71G1Ig,73330,25.306178,73331.0,44.202560,45691.0,166.877640,30438.0,276.295206,58336.0,...,73330.0,73330.0,NaN,NaN,bus_stop,NaN,NaN,NaN,"""bus""=>""yes"",""public_transport""=>""platform""","Nearby,highway is bus_stop,bus is yes,public_transport is platform,"
3,000oRp3E5uFaK1UjpjfCDA,65415,9.594348,49496.0,12.858370,56793.0,23.749958,827.0,27.180638,56759.0,...,826.0,827.0,NaN,NaN,traffic_signals,NaN,NaN,NaN,NaN,"Nearby,highway is traffic_signals,"
4,000q_gZY_eJyop5akxKjjA,44244,53.613539,35488.0,53.882897,44242.0,252.151236,35486.0,293.183568,44282.0,...,35498.0,35488.0,อิสรภาพ39,NaN,NaN,NaN,NaN,NaN,"""shop""=>""convenience""","Nearby,name is อิสรภาพ39,shop is convenience,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884115,zZzwpESoUO9lwq_WIF9bmg,73327,174.724397,73329.0,188.890789,73328.0,190.624950,73333.0,251.807144,43629.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884116,ZzZXZKTEd_YiOFo9CDChgA,14828,129.643955,28702.0,146.434939,74206.0,155.462364,45355.0,405.171300,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
884117,zZZykOeloUZ6-f-4R7tdtg,48327,52.521084,40963.0,133.839863,16704.0,176.684535,66564.0,182.706714,1316.0,...,48330.0,48327.0,Stameeneehoi,NaN,NaN,NaN,NaN,NaN,"""addr:street""=>""ถนนประเสริฐมนูกิจ"",""amenity""=>""restaurant""","Nearby,name is Stameeneehoi,amenity is restaurant,"
884118,zzZytpKByu5aiE_SgGWCVA,64119,34.255010,47849.0,57.774322,48248.0,78.100325,47848.0,121.275198,54040.0,...,47852.0,47849.0,NaN,NaN,NaN,NaN,NaN,NaN,"""name:en""=>""Seven eleven"",""shop""=>""supermarket""","Nearby,name:en is Seven eleven,shop is supermarket,"


In [67]:
# df1.to_excel(best_poi_path)

In [67]:

poi_df_nec.columns

Index(['Unnamed: 0', 'id', 'name', 'barrier', 'highway', 'address', 'place',
       'man_made', 'other_tags', 'poi_prompt'],
      dtype='object')

In [68]:
# desc_pd: file,prompt insert poi_prompt
# df1 : image_key, bestpoi
# poi_df_nec: FID ,poi_prompt
def connect(x):
    image_file_name = x[0]
    image_name =image_file_name.split('.')[0]
    best_poi_row = df1.loc[df1['img_key']==image_name]['best_poi']
    best_poi = None
    if best_poi_row.notna().any():
        best_poi=int(best_poi_row.iloc[0])
        poi_prompt = poi_df_nec.loc[poi_df_nec['id']==best_poi]['poi_prompt'].iloc[0]
        return poi_prompt
        # print(poi_prompt)
    # print(best_poi)
    # print("\n")
    

    # print(image_name)

In [69]:
result2 = desc_pd.apply(lambda x: connect(x), axis=1,result_type='expand')
result2

0          None                                                               
1          None                                                               
2          None                                                               
3          None                                                               
4          None                                                               
           ...                                                                
1005899    None                                                               
1005900    Nearby,highway is motorway_junction,                               
1005901    Nearby,highway is bus_stop,bus is yes,public_transport is platform,
1005902    Nearby,amenity is fuel,                                            
1005903    Nearby,barrier is gate,                                            
Length: 1005904, dtype: object

In [70]:
desc_pd['poi_prompt'] = result2

In [71]:
desc_pd

,key,prompt,poi_prompt
0,--9rgYB2ZCHkuLTrlOlplA,"there is a small building with a fence and a fire hydrant, a detailed matte painting inspired by Gao Cen, behance, paris school, modern lush condo as shopfront, ouchh and and innate studio, lecture halls and gambling dens",None
1,--WYjbVJIq5eUrNyjjxAew,"there is a narrow street with a white brick wall and a blue and white wall, a detailed matte painting inspired by Albert Anker, pixiv, barbizon school, white block fence, found on google street view, narrow footpath",None
2,-0CVvVBkckomDYWJLelOHg,"cars are driving on the road near a city with tall buildings, a tilt shift photo inspired by Ned M. Seidler, featured on pexels, hypermodernism, digital billboard in the middle, found on google street view, electronic billboards",None
3,-0Ti_T4nO59b0V-lrzkGUw,"arafed street with a fence and a car parked on the side, a matte painting inspired by Thomas Struth, shutterstock contest winner, barbizon school, unobstructed road, google maps street view, narrow footpath",None
4,-1Pca6jyCveNvlqippwRaQ,"there is a bus that is driving down the street in the city, a tilt shift photo inspired by Gao Cen, polycount, paris school, lecture halls and gambling dens, narrow footpath, unobstructed road",None
...,...,...,...
1005899,lWYx1OMYiv0tIn8aC0qFiQ,"arafed view of a highway with a truck driving on it, a matte painting by Gao Cen, behance, tonalism, heavy rain approaching, dark cloudy sky, bad weather approaching",None
1005900,trlZkSgxggFMxu2_GGBcRA,"arafly shot of a highway with cars and a bridge, a matte painting by Kume Keiichiro, tumblr, video art, car on highway, dashcam footage, highway","Nearby,highway is motorway_junction,"
1005901,qW-9JShsv4O6tMej0p1aWQ,"cars driving on a highway with a bridge in the background, a matte painting inspired by Friedrich Traffelet, polycount, superflat, dash cam, dashcam footage, dash cam footage","Nearby,highway is bus_stop,bus is yes,public_transport is platform,"
1005902,WgA-pNLZz10kIsqAt7mFEg,"arafed view of a car driving down a street with a bridge, a screenshot by Rube Goldberg, featured on reddit, superflat, dashcam footage, car jumping, dash cam footage","Nearby,amenity is fuel,"


In [73]:
desc_pd.to_excel(desc_poi_path)